## NOTEBOOK OBJECTIVE
The Objective of this Notebook is to extract a set of frames from a specific (what we assume is a standard professional tennis match), compute the frame specific bounding box co-ordinates and use the same in the Bounce Detection Model.

We will be using the weights from the best peforming YOLOv4-tiny model:Module2_Step2g.

Wherever there is a comment for ## UPDATE the code (often a path) needs to be updated.

Libraries used:
1. os
2. shutil
3. watermark

Steps include:
1. STEP1: Mounting Drive
2. STEP2: Installing & Importing Libraries
3. STEP3: Put zipped files into 1 folder and create text file with frame path
4. STEP4: Clone Darknet Repo and save it in Drive
5. STEP5: Compile the Make file
6. STEP6: Copy & Modifications to specific system files
7. STEP7: Run the process.py python script to create the train.txt & test.txt files inside the data folder
8. STEP8: Test trained model on set of tennis match frames to get bounding box co-ordinates
9. STEP9: Dependencies






Inputs will include the following:
1. Set of Video frames, in a set of zipped video frames

Outputs will include the following:
1. Set of Text files containing Tennis ball bounding box co-ordinates, for each frame 

Sources:
1. https://stackoverflow.com/questions/60413212/yolo-darknet-how-to-detect-a-whole-directory-of-images



## STEP1: Mounting Drive

In [ ]:
# Mount drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## STEP2: Installing & Importing Libraries

In [ ]:
## install libraries
!pip install watermark

In [ ]:
import os
import shutil



## Ensure the following folder structure in the working directory where this notebooks is stored -->

-Module3_Bounce_Detection

--input_frames

--labeled_frames

## STEP3: Put zipped files into 1 folder and create text file with frame path

Download the 4 zipped files 1_to_2000.zip, 2001_to_4000.zip, 4001_to_6000.zip and 6001_to_8462.zip from the gdrive folder https://drive.google.com/drive/folders/1-skl0_iiKYZDJPM6hhFxCqmW-cgqGh3q?usp=sharing to '/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/input_frames'.



In [ ]:
%%time
# Unzipping the 4 zipped folders into a single folder
import zipfile

zippped_file_lst=['/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/input_frames/1_to_2000.zip',
                  '/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/input_frames/2001_to_4000.zip',
                  '/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/input_frames/4001_to_6000.zip',
                  '/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/input_frames/6001_to_8462.zip']

unzipped_folder_pth='/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames'


for zp_fl_pth in zippped_file_lst:
  shutil.unpack_archive(zp_fl_pth, unzipped_folder_pth)


print("No of Files in unzipped video frame folder",len(os.listdir(unzipped_folder_pth)))

No of Files in unzipped video frame folder 8463
CPU times: user 8.46 s, sys: 2.09 s, total: 10.5 s
Wall time: 53 s


In [ ]:
# Set Directory path

os.chdir(unzipped_folder_pth) ## UPDATE

## Extract file names to a text file
!ls > /content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/filenames.txt ## UPDATE


In [ ]:
# Clean up text file + add path string

# Open up text file
filepath = "/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/filenames.txt" ## UPDATE
with open(filepath) as fp:
    lines = fp.read().splitlines()

print(lines[:10])
print(len(lines))

# Remove unwanted file/ folder 
lines.remove('__MACOSX')
print(lines[:10])
print(len(lines))

# read each line and add the unzipped_folder_pth before the image file name 
values=[]
for fil in lines:
  val=unzipped_folder_pth +"/"+fil
  values.append(val)

print(values[:10])

pth='/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/img_file.txt'
with open(pth, 'w') as filehandle:
    for listitem in values:
        filehandle.write('%s\n' % listitem)

with open(pth) as f:
        testList = f.readlines()

print(len(testList))
print(testList[:3])



['image1000.jpg', 'image1001.jpg', 'image1002.jpg', 'image1003.jpg', 'image1004.jpg', 'image1005.jpg', 'image1006.jpg', 'image1007.jpg', 'image1008.jpg', 'image1009.jpg']
8463
['image1000.jpg', 'image1001.jpg', 'image1002.jpg', 'image1003.jpg', 'image1004.jpg', 'image1005.jpg', 'image1006.jpg', 'image1007.jpg', 'image1008.jpg', 'image1009.jpg']
8462
['/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames/image1000.jpg', '/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames/image1001.jpg', '/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames/image1002.jpg', '/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames/image1003.jpg', '/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames/image1004.jpg', '/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames/image1005.jpg', '/content/drive/MyDrive

## STEP4: Clone Darknet Repo and save it in Drive

In [ ]:
# Set Directory path

path="/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny" ## UPDATE
os.chdir(path) # Change Path

In [ ]:
# Clone Darknet Repo -clone the repo for each batch to refresh files
try:
  shutil.rmtree(path+"/darknet")
  !git clone https://github.com/AlexeyAB/darknet
except:
  # shutil.rmtree(path+"/darknet")
  !git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.02 MiB | 7.24 MiB/s, done.
Resolving deltas: 100% (10356/10356), done.
Checking out files: 100% (2050/2050), done.


In [ ]:
# Verify CUDA version
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


## STEP5: Compile the Make file

In [ ]:
%%time
# Compile the Darknet framework in order to use the related files for object detection model
path="/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/darknet" ## UPDATE
os.chdir(path) # Change Path


!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

CPU times: user 16.9 ms, sys: 24.5 ms, total: 41.5 ms
Wall time: 569 ms


In [ ]:
%%time
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old ==

In [ ]:
# verify installation
!./darknet

usage: ./darknet <function>


In [ ]:
# # Run following code if you get permission denied error
# !sudo chmod +x darknet
# !./darknet

## STEP6: Copy & Modifications to specific system files

We have the following system files:
1. Customized configuration file
2. Customized obj.data file
3. Customized obj.names file
4. process.py file
5. YOLOv4-tiny pre trained best weights from Module2-Step2g
6. Text file containing path to images on which we will detect the tennis ball bounding box co-ordinates.

Key steps include:
1. We keep the network resolution at 1664 width and height
2. change max_batches to 2000 as we have only 1 class OR atleast 6000 OR size of training images. We keep 6000 as rest approches would result in a lower max_batches.
3. change steps to 4800,5400 (80% & 90% of max_batches)
4. In the 2 convolutional layers before the YOLO layers, change filters =(x+5)*3=18 as the number of class =1
5. Change classes =1 in the 2 YOLO layers

The obj.data file needs to have the following content:

classes = 1

train  = data/train.txt

valid  = data/test.txt

names = data/obj.names

backup = /content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/training




The obj.names needs to have the following content:

ball

No change needed in the process.py file

The original obj.data, obj.names and process.py file can be downloaded from (https://github.com/techzizou/yolov4-tiny-custom_Training/tree/main/yolov4-tiny)


In [ ]:
# copy the custom cfg file from the drive to the darknet/cfg folder
!cp /content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/yolov4-tiny-custom.cfg ./cfg  ## UPDATE

In [ ]:
## Update cfg file
%cd cfg
!sed -i 's/width=416/width=1664/' yolov4-tiny-custom.cfg ## UPDATE
!sed -i 's/height=416/height=1664/' yolov4-tiny-custom.cfg ## UPDATE
!sed -i 's/max_batches = 6000/max_batches = 6000/' yolov4-tiny-custom.cfg ## UPDATE 2000* number of classes, not less than number of training images and not less than 6000
!sed -i 's/steps=4800,5400/steps=4800,5400/' yolov4-tiny-custom.cfg ## UPDATE 80% & 90% of max_batches
%cd ..

/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/darknet/cfg
/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/darknet


In [ ]:
# copy the obj.names and obj.data files so that they are now in /darknet/data/ folder
!cp /content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/obj.names ./data 
!cp /content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/obj.data  ./data 

In [ ]:
#copy the process.py file from the drive to the darknet directory
!cp /content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/process.py ./ 

In [ ]:
# run process.py ( this creates the train.txt and test.txt files in our darknet/data folder )
!python process.py

# list the contents of data folder to check if the train.txt and test.txt files have been created 
!ls data/

/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/darknet
9k.tree     eagle.jpg	 imagenet.labels.list	   obj.names	     test.txt
coco9k.map  giraffe.jpg  imagenet.shortnames.list  openimages.names  train.txt
coco.names  goal.txt	 labels			   person.jpg	     voc.names
dog.jpg     horses.jpg	 obj.data		   scream.jpg


## STEP8: Test trained model on set of tennis match frames to get bounding box co-ordinates

In [ ]:
## set your custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-tiny-custom.cfg ## UPDATE
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-tiny-custom.cfg ## UPDATE
%cd ..

/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/darknet/cfg
/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/darknet


In [ ]:
%%time
## Object detection on multiple images
## UPDATE change the path to the best weights from Module2_Step2g 
## Best weights are at : /content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/SAVED_training_weights/training_proam_1008_aug_1664res/yolov4-tiny-custom_best.weights
## file path is at : /content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/img_file.txt


!./darknet detector test data/obj.data cfg/yolov4-tiny-custom.cfg /content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module2_Object_Detection_Yolov4_tiny/SAVED_training_weights/training_proam_1008_aug_1664res/yolov4-tiny-custom_best.weights < /content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/img_file.txt -ext_output -dont_show -save_labels

Streaming output truncated to the last 5000 lines.
Enter Image Path:  Detection layer: 30 - type = 28 
 Detection layer: 37 - type = 28 
/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames/image7268.jpg: Predicted in 43.032000 milli-seconds.
Enter Image Path:  Detection layer: 30 - type = 28 
 Detection layer: 37 - type = 28 
/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames/image7269.jpg: Predicted in 43.015000 milli-seconds.
Enter Image Path:  Detection layer: 30 - type = 28 
 Detection layer: 37 - type = 28 
/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames/image726.jpg: Predicted in 43.098000 milli-seconds.
Enter Image Path:  Detection layer: 30 - type = 28 
 Detection layer: 37 - type = 28 
/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames/image7270.jpg: Predicted in 43.027000 milli-seconds.
Enter Image Path:  Detection layer: 30 - type 

In [ ]:
unzipped_folder_pth='/content/drive/MyDrive/CAPSTONE/CAPSTONE_FINAL/Module3_Bounce_Detection/labeled_frames' ## UPDATE
print("Num of Files in unzipped video frame folder",len(os.listdir(unzipped_folder_pth)))

Num of Files in unzipped video frame folder 16925


## STEP9: Dependencies

In [ ]:
# Dependencies
%reload_ext watermark
%watermark
%watermark --iversions

Last updated: 2022-04-22T19:53:22.048727+00:00

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 5.5.0

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

google : 2.0.3
IPython: 5.5.0

